# Working with Agents

An agent is a language model that decides on what actions to take for a given prompt and in which order.

In today's example, we are going to an agent that decides whether to retrieve information from an LLM's general knowledge (by passing a prompt to the underlying LLM) or by calling an API.

## The innate staleness of LLM data

One of the downsides of LLMs is that their training data only captures a snapshot of human knowledge up to a certain point in time. This cutoff creates an inherent staleness in the data, as the model cannot reflect information beyond its last update. For instance, let's try asking an LLM what the current weather in New York is:

In [1]:
from langchain_ollama import ChatOllama

# Context window of mistral is rougly 8k
# https://api.python.langchain.com/en/latest/llms/langchain_community.llms.ollama.Ollama.html#langchain_community.llms.ollama.Ollama
llm = ChatOllama(model="mistral",
            # temperature=0.9, # Increasing the temperature will make the model answer more creatively. (Default: 0.8),
            # num_ctx=4096  # Default is 2048
            )

llm.invoke("Please tell me the current weather in New York city for today").content

" I'm an AI and cannot access real-time data. However, as of my last update, the forecast for New York City on August 16, 2023 is sunny with a high of 81°F (27°C) and a low of 69°F (20°C). Please check a reliable weather source for the most accurate and up-to-date information."

Clearly this demonstrates the above point regarding the cutoff. There is also no built-in way to check online weather sources. Repeating the above query can yield different results too - with a high likelyhood of hallucinations. This is where Agents can help us when it comes to using alternate sources of information - for instance the current weather in a certain location.


## API setup

Let us first create a function that performs an API call to the weatherapi, which provides realtime weather data for a specified location.

In [2]:
import requests
# Note: for demo purposes - throwaway key. In practice you should NEVER expose API keys like this (use ie env variables)
FREE_WEATHER_API_KEY="aa96b0814fc94f09847100745241005"

def make_weather_api_call(location: str):
    response = requests.get(f"https://api.weatherapi.com/v1/current.json?key={FREE_WEATHER_API_KEY}&q={location}")
    if (response.status_code != 200):
        raise ValueError("Could not obtain the weather for the current parameters")
    return response.json()

Let's try this out to get the current London weather and inspect the resultant object

In [3]:
make_weather_api_call("London")

{'location': {'name': 'London',
  'region': 'City of London, Greater London',
  'country': 'United Kingdom',
  'lat': 51.52,
  'lon': -0.11,
  'tz_id': 'Europe/London',
  'localtime_epoch': 1726589942,
  'localtime': '2024-09-17 17:19'},
 'current': {'last_updated_epoch': 1726589700,
  'last_updated': '2024-09-17 17:15',
  'temp_c': 21.0,
  'temp_f': 69.8,
  'is_day': 1,
  'condition': {'text': 'Partly cloudy',
   'icon': '//cdn.weatherapi.com/weather/64x64/day/116.png',
   'code': 1003},
  'wind_mph': 10.1,
  'wind_kph': 16.2,
  'wind_degree': 66,
  'wind_dir': 'ENE',
  'pressure_mb': 1029.0,
  'pressure_in': 30.39,
  'precip_mm': 0.0,
  'precip_in': 0.0,
  'humidity': 46,
  'cloud': 25,
  'feelslike_c': 21.0,
  'feelslike_f': 69.8,
  'windchill_c': 19.4,
  'windchill_f': 66.9,
  'heatindex_c': 19.4,
  'heatindex_f': 66.9,
  'dewpoint_c': 10.5,
  'dewpoint_f': 50.9,
  'vis_km': 10.0,
  'vis_miles': 6.0,
  'uv': 5.0,
  'gust_mph': 13.4,
  'gust_kph': 21.6}}

Clearly there is a lot of information returned here. Let's refine the function to return only the temperature in Celsius in a nicely formatted string.

In [4]:
def make_weather_api_call(location: str):
    response = requests.get(f"https://api.weatherapi.com/v1/current.json?key={FREE_WEATHER_API_KEY}&q={location}")
    if (response.status_code != 200):
        raise ValueError("Could not obtain the weather for the current parameters")
    return f'The current weather in {location} is {response.json()["current"]["temp_c"]} degrees celsius.'

make_weather_api_call("London")

'The current weather in London is 21.0 degrees celsius.'

The solution is to configure an Agent to determine whether a prompt should be passed to the weather API, or to an underlying LLM for general information. We can use our previously defined LLM object to handle any non-weather related prompts.

## Configuring Agents

As previously mentioned, an agent is an LLM configured for making decisions on what actions to execute. Agents work with interfaces known as **tools** to interact with different functions. Our agent is expected to either make a call to an LLM, or to our weather api function - we therefore need to define two tools for the agent to use:

1. A weather API tool
2. A general-purpose LLM invocation tool

We will begin by creating an Agent LLM object that is responsible for managing these tools. The model we supply must be compatible with tooling. Here we will use `llama3.1` for best results.

In [5]:
from langchain_ollama import ChatOllama


# We set the temperature to 0 for deterministic responses (no creative freedom).
# We must set a model which supports tooling, see https://ollama.com/library for models tagged with "tools"
agent_llm = ChatOllama(model="llama3.1", temperature=0)

We will now create an array of tools for the model to use. We can use langchain's tool decorator with functions to denote them as tools. We will also use `typing.Annotated` to pass descriptions for each argument. Finally, all our tools need a docstring with a description that serves as a prompt to the agent LLM to help it decide whether it is appropriate to call or not. This is a crucial point where good prompt engineering is essential.

In [7]:
from langchain_core.tools import tool, ToolException
from typing import Annotated
from langchain.agents import AgentExecutor


@tool
def general_knowledge_tool(prompt: Annotated[str, "The prompt to pass to the general knowledge LLM"]) -> str:
    """
    Useful when you need to answer any questions that are not related to the current real-time weather.
    You must pass the exact input without changing anything to the function. For instance, if the input is

        "Tell me about the Americal Civil War"

    You must pass the argument

        "Tell me about the Americal Civil War"

    to the function
    """
    try:
        return llm.invoke(prompt).content
    except Exception as e:
        raise ToolException(e)

@tool
def weather_report_api(location: Annotated[str, "The location for which to get the weather"]) -> float:
    """
    Useful when you need to answer questions related to the current real-time weather in a certain location. 
    This tool can only be used to get current real-time weather information. It does not have any information on average, or
    historical weather at a certain location. The tool returns a single double value representing the temperature at a location
    in degrees celsius. The only argument it takes is a string name of the location. For instance, if the question is:

        "What's the weather like in Santa Monica"

    You must pass the string "Santa Monica" to the tool and nothing else.
    """
    try:
        return make_weather_api_call(location)
    except Exception as e:
        raise ToolException(e)

tools = [general_knowledge_tool, weather_report_api]

# Print tool
print(general_knowledge_tool)


name='general_knowledge_tool' description='Useful when you need to answer any questions that are not related to the current real-time weather.\nYou must pass the exact input without changing anything to the function. For instance, if the input is\n\n    "Tell me about the Americal Civil War"\n\nYou must pass the argument\n\n    "Tell me about the Americal Civil War"\n\nto the function' args_schema=<class 'langchain_core.utils.pydantic.general_knowledge_tool'> func=<function general_knowledge_tool at 0x109aa7ba0>


We have defined two json tool objects above and stored them in an array. Note how the description prompt plays a huge role in the decision making for the agent.

We will now bind these tools to our llm agent, and print the result of invoking it with some examples. We will check two properties of the response, namely the `tool_calls` and `content` properties (which we have used before for getting the llm response).

In [8]:
from langchain.agents import AgentExecutor, create_tool_calling_agent

agent_llm = agent_llm.bind_tools([general_knowledge_tool, weather_report_api])

result = agent_llm.invoke("What's the current weather in Chicago?")
print(result.content)
print(result.tool_calls)

result = agent_llm.invoke("Tell me about Javascript")
print(result.content)
print(result.tool_calls)

# Invalid tool_calls will be listed under .invalid_tool_calls


[{'name': 'weather_report_api', 'args': {'location': 'Chicago'}, 'id': 'a0022eef-5bf6-469a-bd57-b725f58b38d5', 'type': 'tool_call'}]

[{'name': 'general_knowledge_tool', 'args': {'prompt': 'Tell me about Javascript'}, 'id': 'b6c85ded-f96b-4fb9-9717-a5506d3b977e', 'type': 'tool_call'}]


In the printouts for `tool_calls` we can see that the agent successfully deduces which tool it needs to call, and correctly formulates the arguments such that they obey the schemas specified. Surprisingly, the `content` property is empty - it doesn't contain the outputs from the functions themselves. The agent is responsible purely for deciding what actions to take, but by itself it will not call the functions.

To call the functions themselves, we can create an executor class which wraps our agent, and its tools, and calls the respective functions with their arguments.

In [9]:
class AgentExecutor:
    def __init__(self, agent, tools, verbose: bool = False):
        self.agent = agent
        self.tools = {tool.name: tool for tool in tools} # Create a tool map with the tool name as key
        self.verbose = verbose

    def invoke(self, inp: str):
        result = self.agent.invoke(inp)

        if(len(result.tool_calls) == 0):
            raise ValueError("Could deduce tool invocation")
        
        function_name = result.tool_calls[0]["name"]
        function_args = result.tool_calls[0]["args"]
        try:
            if(self.verbose):
                print(f"================= Calling {function_name} =================")
            return self.tools[function_name].invoke(input=function_args)
        except Exception as e:
            raise ValueError("Could not invoke tool")
            
        

With our simple agent executor ready, let's begin using it:

In [10]:

agent_executor = AgentExecutor(agent_llm, tools, verbose=True)

print(agent_executor.invoke("What's the current weather in San Francisco?"))

print(agent_executor.invoke("Tell me a joke"))

================= Calling weather_report_api =================
The current weather in San Francisco is 16.7 degrees celsius.
================= Calling general_knowledge_tool =================
 Why was the math book sad? Because it had too many problems.

(Bonus joke for physics enthusiasts: Why did the physics major go to the bar? To find his missing electron.)
